<a href="https://colab.research.google.com/github/bckim9489/agent_practice/blob/main/agent_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Common Setup

참고 : 아래 Secret으로 Key 등록해서 실습함
1.   LLM API Key (GPT, claude 등) : OPENAI-KEY 로 등록했음
2.   Tavily Serach API Key (https://app.tavily.com/home) : TAVIL-KEY 로 등록했음



In [ ]:
!pip -q install -U "requests==2.32.5" "pydantic==2.12.3" "httpx==0.28.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip -q install -U langgraph langchain-openai langchain-community wikipedia numexpr tavily-python

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
import os
from langchain_openai import ChatOpenAI
from google.colab import userdata

# LLM API Key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI-KEY")

# Tools
# Search API Key
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY-KEY")

llm = ChatOpenAI(model="gpt-4o", temperature=0)


## ㄴ 0. TEST

In [ ]:
import requests, httpx, pydantic
import langchain_community, langgraph
from langgraph.graph import StateGraph

print("requests:", requests.__version__)
print("httpx:", httpx.__version__)
print("pydantic:", pydantic.__version__)
print("langchain-community:", langchain_community.__version__)
print("LangGraph import OK")
print("StateGraph:", StateGraph)

requests: 2.32.4
httpx: 0.28.1
pydantic: 2.12.3
langchain-community: 0.4.1
LangGraph import OK
StateGraph: <class 'langgraph.graph.state.StateGraph'>


## ㄴ 1. Tools

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_core.tools import tool
import numexpr

# Wikipedia tool
wiki = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=2000)
)

# 기존 wiki wrapper 재사용
wiki_lookup_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=4000)

@tool
def docstore_lookup(title: str) -> str:
    """
    Lookup a specific document by title from the docstore (Wikipedia).
    Input should be a precise title or entity name.
    """
    return wiki_lookup_wrapper.run(title)

# Tavily Search tool
search_tool = TavilySearchResults(max_results=5)

# Calculator tool (llm-math 대체)
@tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression.
    Input must be a valid expression like '345*872' or '12/4+9'.
    """
    try:
        return str(numexpr.evaluate(expression))
    except Exception as e:
        return f"Error: {e}"


tools = [wiki, calculator, search_tool, docstore_lookup]
print("=== Tools ===")
for t in tools:
    print("-", t.name)

=== Tools ===
- wikipedia
- calculator
- tavily_search_results_json
- docstore_lookup


/tmp/ipython-input-1012767205.py:25: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=5)


#
----------------------------------------------------------

# [Gen 1] Prompt-driven Tool-Using Agents (ReAct-based Agents)
> **LLM**이 매 순간 다음 행동을 **즉흥 결정**
*   **LLM-Driven Agents**
*   LLM이 생각(Reasoning) 하고
*   외부 도구를 행동(Act) 으로 호출하며
*   그 결과를 다시 보고 추론을 이어가는 "Tool-using LLM Loop"
*   2022 ~ 2024




## ㄴ 1. Zero-Shot ReAct
*   기억 없음(Stateless)
*   **LLM + Tools**





### ㄴ 1.1 Description

#### ㄴ 1.1.1 Workflow


```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
LLM: 질문 해석 (Thought / Reasoning)
  ↓
Tool 필요 여부 판단
  ├─ 필요 없음 → 바로 답 생성
  └─ 필요 있음 → Tool 선택 (Action)
                      ↓
                Tool 실행 (Python / API / DB 등)
                      ↓
                결과 반환 (Observation)
                      ↓
LLM이 결과 반영하여 다시 Reasoning
  ↓
(필요 시 Thought → Action → Observation 반복)
  ↓
Final Answer 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시

```

#### ㄴ 1.1.2 Inner ReAct Loop



``` javascript
while 문제 해결 전:
    Thought      → 어떻게 풀지 스스로 판단
    Action       → 사용할 Tool 선택
    Observation  → Tool 실행 결과 받기
    Thought      → 결과 보고 다음 행동 결정
```



### ㄴ 1.2 Code

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

In [ ]:
q = """위키피디아에서 찾기 좋은 '검색어(제목형 키워드)'를 3개 만들어서,
그 키워드로 Wikipedia tool을 사용해 검색하고 근거를 요약해줘.
주제: Pinus densiflora 옮겨심기(이식) 적기"""

In [ ]:
result = agent.invoke({"messages": [("user", q)]})

print(result["messages"][-1].content)

### ㄴ 1.3 Verbose

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

for step in agent.stream(
    {"messages": [("user", q)]},
    stream_mode="values",
):
    msg = step["messages"][-1]

    # LLM이 생각/결정한 내용
    if isinstance(msg, AIMessage):
        print("\n AI MESSAGE")
        print(msg.content)

        if getattr(msg, "tool_calls", None):
            print(" TOOL CALL:", msg.tool_calls)

    # Tool 실행 결과
    elif isinstance(msg, ToolMessage):
        print("\n TOOL RESULT")
        print("tool:", msg.name)
        print(msg.content[:1000])  # 너무 길면 앞부분만


## ㄴ 2. Conversational ReAct
*   메모리 사용
*   **LLM + Tools + Checkpointer**



### ㄴ 2.1 Description

#### ㄴ 2.1.1 Workflow

```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
이전 대화 기록(Memory / State) 로드
  ↓
LLM: 현재 질문 + 대화 히스토리 함께 해석
  ↓
사용자의 의도 파악 (맥락 기반 Reasoning)
  ↓
Tool 필요 여부 판단
  ├─ 필요 없음 → 바로 답 생성
  └─ 필요 있음 → Tool 선택 (Action)
                      ↓
                Tool 실행 (Python / API / DB 등)
                      ↓
                결과 반환 (Observation)
                      ↓
LLM이 결과 + 이전 대화 맥락을 반영하여 다시 Reasoning
  ↓
(필요 시 Thought → Action → Observation 반복)
  ↓
Final Answer 생성
  ↓
대화 Memory(State)에 이번 메시지 저장
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시
```

#### ㄴ 2.1.2 Inner ReAct Loop

```javascript
while 문제 해결 전:
    Load Memory        → 이전 대화 불러오기
    Thought            → 맥락 기반으로 판단
    Action             → Tool 필요하면 실행
    Observation        → Tool 결과 받기
    Update Memory      → 새 정보 저장
```

### ㄴ 2.2 Code

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()
agent = create_react_agent(llm, tools, checkpointer=checkpointer)

In [ ]:
config = {"configurable": {"thread_id": "chat-001"}}
q1 = "내 출생년도는 1994년이야. 지금은 계산하지 말고 이 정보만 기억해."
q2 = "그럼 내가 2026년에 몇 살인지 계산기를 꼭 사용해서 나이를 계산해서 알려줘"
q3 = "올해는 무슨 해이며 나는 무슨 띠인지 알려줘"

In [ ]:
result = agent.invoke({"messages": [("user", q1)]}, config=config)
print(result["messages"][-1].content)
result = agent.invoke({"messages": [("user", q2)]}, config=config)
print(result["messages"][-1].content)
result = agent.invoke({"messages": [("user", q3)]}, config=config)
print(result["messages"][-1].content)


### ㄴ 2.3 Verbose

#### ㄴ 2.3.1 Test Setup

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage
import time
from openai import RateLimitError

from langchain_core.messages import AIMessage, ToolMessage

def stream_with_retry(agent, payload, config, stream_mode="values", max_retries=6):
    """
    agent.stream(...) 를 RateLimitError(429) 발생 시 backoff 재시도.
    """
    attempt = 0
    while True:
        try:
            for step in agent.stream(payload, config=config, stream_mode=stream_mode):
                yield step
            return  # 정상 종료
        except RateLimitError as e:
            attempt += 1
            if attempt > max_retries:
                raise

            # 간단 backoff (점점 더 기다림)
            wait = min(20, 1.5 * attempt)
            print(f"\n RateLimit(429). retry in {wait:.1f}s ...")
            time.sleep(wait)



def run_verbose(question: str, label: str, thread_id: str):
    print("\n" + "="*20 + f" {label} (thread_id={thread_id}) " + "="*20)
    print("USER:", question)

    cfg = {"configurable": {"thread_id": thread_id}}

    payload = {"messages": [("user", question)]}

    last_ai = None
    for step in stream_with_retry(agent, payload, cfg, stream_mode="values"):
        msg = step["messages"][-1]

        if isinstance(msg, AIMessage):
            last_ai = msg
            if msg.content:
                print("\nAI:", msg.content)
            if getattr(msg, "tool_calls", None):
                print("tool_calls:", msg.tool_calls)

        elif isinstance(msg, ToolMessage):
            print("\nTOOL RESULT:", msg.name)
            print(msg.content[:800])

    return last_ai.content if last_ai else None

#메모리 사용하는지 A/B 테스트
def ab_test(q1: str, q2: str, q3:str, thread_a: str="mem-A", thread_b: str="mem-B", thread_c: str="mem-C"):
    print("\n\n" + "#"*10 + " A/B MEMORY TEST START " + "#"*10)

    # A: q1 using thread_id a
    run_verbose(q1, "A-1: seed memory (q1)", thread_a)


    # B: q2 using thread_id b
    run_verbose(q2, "B-1: q2 only (no memory)", thread_b)


    # B: q3 using thread_id c
    run_verbose(q3, "C-1: q3 only (no memory)", thread_c)

    print("\n" + "#"*10 + " A/B MEMORY TEST END " + "#"*10)


#### ㄴ 2.3.2 Test A (같은 thread_id)

*   Q1을 thread_id A 에 저장
*   Q2를 thread_id A 에 질의
----------------------------
| Q2에서 메모리에 저장한 Q1을 활용하여 답변할 것으로 예상

In [ ]:
# 같은 thread_id
run_verbose(q1, "Q1", "chat-001")
run_verbose(q2, "Q2", "chat-001")
run_verbose(q3, "Q3", "chat-001")

#### ㄴ 2.3.3 TEST B (다른 thread_id)

*   Q1을 thread_id A 에 저장
*   Q2를 thread_id B에 질의
----------------------------
| Q2에서 질의 에 대한 정보 요구 예상



In [ ]:
ab_test(q1, q2, q3, thread_a="chat-003", thread_b="chat-004", thread_c="chat-005")

## ㄴ 3. Search-augmented ReAct (Self-ask with search)

*   질문 분해 중심
*   **RAG와 매우 유사**
*   사실상 Self-Ask = 초기 형태의 RAG



### ㄴ 3.1 Description

#### ㄴ 3.1.1. Workflow



```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph / Agent Runtime 시작
  ↓
LLM이 질문을 "하위 질문(Sub-questions)"으로 분해
  ↓
각 하위 질문을 Search Tool로 순차 조회
  ↓
검색 결과(Observation)를 모아 중간 결론 생성
  ↓
필요하면 추가 하위 질문 생성 → 다시 Search
  ↓
모든 정보가 충분해지면 최종 답 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시
```



#### ㄴ 3.1.2 Inner ReAct Loop

```javascript
while 답을 만들 정보가 부족하면:
    Follow-up Question 생성
    Search Tool 실행
    Observation 수집
    현재까지의 사실 정리
```

### ㄴ 3.2 Code

In [ ]:
from langchain_core.messages import SystemMessage

# Self-Ask의 핵심 포맷을 강제
SELF_ASK_POLICY = SystemMessage(content="""
You are a Self-Ask-with-Search agent.

Hard rules:
- For EACH follow-up question, you MUST call the tavily_search_results_json tool.
- After the tool returns, you MUST write an Intermediate answer that uses the tool result.
- Intermediate answer must include 2-3 bullet points and mention the source titles (from tool results).
- Do NOT write placeholders like "...". If info is missing, do another search.

Output format (exact):
Follow-up 1: ...
Intermediate answer 1:
- ...
- ...
Sources: <title1>, <title2>

Follow-up 2: ...
Intermediate answer 2:
- ...
- ...
Sources: ...

Final answer: ...
""")


In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

In [ ]:
q = """AI Agent를 3문장으로 설명해줘.
단, 2025~2026년 기준으로 최신 경향을 반영하기 위해 반드시 tavily_search_results_json을 최소 2번 사용하고,
각 검색 결과를 근거로 Intermediate answer를 작성한 뒤 Final answer를 써."""

In [ ]:
config = {"configurable": {"thread_id": "self-ask-01"}}

result = agent.invoke({"messages": [SELF_ASK_POLICY, ("user", q)]},config=config)

print(result["messages"][-1].content)

### ㄴ 3.3. Verbose

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

def run_self_ask_verbose(question: str, thread_id="self-ask-03", tool_preview=600):
    cfg = {"configurable": {"thread_id": thread_id}}

    print("\n" + "="*80)
    print("USER:", question)
    print("="*80)

    for step in agent.stream(
        {"messages": [SELF_ASK_POLICY, ("user", question)]},
        config=cfg,
        stream_mode="values",
    ):
        msg = step["messages"][-1]

        # 1) 모델이 말한 텍스트(중간 출력 포함)
        if isinstance(msg, AIMessage):
            if msg.content:  # content가 있는 경우만 출력
                print("\n AI MESSAGE:\n", msg.content)

            # 2) 도구 호출 로그
            if getattr(msg, "tool_calls", None):
                print("\n TOOL CALLS:")
                for tc in msg.tool_calls:
                    print(" -", tc)

        # 3) 도구 결과
        elif isinstance(msg, ToolMessage):
            print("\n TOOL RESULT:", msg.name)
            print(msg.content[:tool_preview])

    print("\n" + "="*80)
    print("END")
    print("="*80)

run_self_ask_verbose(q)


## ㄴ 4. ReAct docstore

*   LLM이 문서 저장소를 스스로 탐색
*   필요한 문서를 조회(Lookup)하여 근거 기반 답변을 생성



### ㄴ 4.1 Description

#### ㄴ 4.1.1 Workflow

```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
LLM: 질문 해석 (문서 탐색 필요 판단)
  ↓
Docstore Search Tool 호출 (Search Action)
  ↓
관련 문서 후보 목록 반환 (Observation)
  ↓
LLM: 어떤 문서를 읽을지 Reasoning
  ↓
Docstore Lookup Tool 호출 (Lookup Action)
  ↓
선택된 문서 실제 내용 반환 (Observation)
  ↓
LLM이 문서 내용을 기반으로 재해석 / 근거 정리
  ↓
(필요 시 Search → Lookup 반복)
  ↓
충분한 근거 확보 후 Final Answer 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시

```

#### ㄴ 4.1.2 Inner ReAct Loop

```javascript
while (답변에 필요한 근거가 충분하지 않다):
    Thought: 질문을 해결하려면 어떤 문서를 찾아야 하는가?
    Action: Search(query)
    Observation: 관련 문서 후보 목록 반환
    Thought: 어떤 문서를 읽어야 하는가?
    Action: Lookup(document_id)
    Observation: 문서 내용 확보
    Thought:이 정보로 답변 가능한가?
            ├─ NO → 다른 문서 다시 Search
            └─ YES → 반복 종료
```

### ㄴ 4.2 Code

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

In [ ]:
q = """
규칙:
- wikipedia 도구를 1회 호출해서 후보 제목을 찾는다.
- 그 다음 docstore_lookup 도구를 반드시 1회 이상 호출한다.
- docstore_lookup 결과를 근거로 최종 답변을 작성한다.

질문: 밍크 선인장(Mammillaria)은 어떤 식물인지 설명해줘.
"""

In [ ]:

config = {"configurable": {"thread_id": "docstore-01"}}
result = agent.invoke({"messages": [("user", q)]},config=config)

print(result["messages"][-1].content)


### ㄴ 4.3 Verbose

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

def run_docstore_verbose(question: str, thread_id="docstore-debug"):
    cfg = {"configurable": {"thread_id": thread_id}}

    print("\n================ USER ================")
    print(question)

    for step in agent.stream(
        {"messages": [("user", question)]},
        config=cfg,
        stream_mode="values",   # 상태 변화 전부 받기
    ):
        msg = step["messages"][-1]

        # LLM이 생각한 내용 (Thought / Action 결정)
        if isinstance(msg, AIMessage):
            if msg.content:
                print("\n AI THOUGHT:")
                print(msg.content)

            if getattr(msg, "tool_calls", None):
                print("\n TOOL CALL:")
                print(msg.tool_calls)

        # Tool 실행 결과 (Observation)
        elif isinstance(msg, ToolMessage):
            print("\n TOOL RESULT:", msg.name)
            print(msg.content[:1000])  # 너무 길어서 제한

    print("\n================ END ================\n")

run_docstore_verbose(q)


#
--------------------------------------------------------------

# [Gen 2] LLM Orchestrated Systems (Post-ReAct Agents)
> Workflow Engine이 **순서를 결정**, LLM은 **필요할 때만 호출**
*   **System-Driven Agents**
*   계획 수립 / 의도 해석   ← LLM
*   실행 흐름은 시스템이 통제 ← Runtime / Graph / Process
*   Tool 실행은 강제된 구조로 수행
*   상태(State)를 외부에서 관리
*   2024 ~ 현재

```javascript
User → Orchestrator → Planner → Executor → Validator → State Update
                        ↓
                      LLM (as function)
```


## ㄴ 1. Plan-Then-Execute (Two-Phase Execution Architecture)
> Plan → **Freeze** Plan → **Execute** Deterministically
>

### ㄴ 1.1 Description

```javascript
Planner LLM → Execution Graph 생성
Execution Engine:
    Step1
    Step2
    Step3
```
*    **Planner(LLM)** 는 계획(어떤 툴을 어떤 순서로 쓸지)를 JSON Plan으로만 만듬 (실행 금지)
*    **Executor(시스템)** 가 계획을 결정론적으로 실행 (툴을 실제로 호출)
*    **Writer(LLM)** 가 결과를 자연어로 정리 및 답변

```javascript
User Input
   ↓
Planner (LLM 1회)
   ↓
Executor (Deterministic Tool Calls)
   ↓
Writer (LLM 정리)
   ↓
End
```

### ㄴ 1.2 Code

#### ㄴ 1.2.1 Tool Registry

In [ ]:
import json
from typing import List, Literal, Dict, Any

# 우리가 Plan에서 쓸 "툴 alias"
TOOL_REGISTRY = {
    "wiki": wiki,
    "search": search_tool,
    "lookup": docstore_lookup,
    "calc": calculator,
}

ToolAlias = Literal["wiki", "search", "lookup", "calc", "final_answer"]

#### ㄴ 1.2.2 Contract

##### ㄴ 1.2.2.1 Plan Schema

In [ ]:
from pydantic import BaseModel, Field

class Step(BaseModel):
    tool: ToolAlias
    input: str

class Plan(BaseModel):
    goal: str
    steps: List[Step] = Field(min_items=1, max_items=6)

/tmp/ipython-input-3491100315.py:9: PydanticDeprecatedSince20: `min_items` is deprecated and will be removed, use `min_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  steps: List[Step] = Field(min_items=1, max_items=6)
/tmp/ipython-input-3491100315.py:9: PydanticDeprecatedSince20: `max_items` is deprecated and will be removed, use `max_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  steps: List[Step] = Field(min_items=1, max_items=6)


##### ㄴ 1.2.2.2 System Prompt

###### ㄴ 1.2.2.2.1 Planner System Prompt

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

PLANNER_SYS = SystemMessage(content=
"""You are a Planner.
Return ONLY valid JSON matching this schema:

{
  "goal": string,
  "steps": [{"tool": "wiki"|"search"|"lookup"|"calc"|"final_answer", "input": string}, ...]
}

Rules:
- Do NOT execute tools.
- Prefer "search" for up-to-date web info when needed.
- Use "wiki" for general grounding.
- Use "lookup" only when you know the exact Wikipedia title/entity.
- Use "calc" only for arithmetic.
- Keep steps minimal (<=4 if possible).
- The last step MUST be "final_answer".
""")

###### ㄴ 1.2.2.2.2 Writer System Prompt

In [ ]:
WRITER_SYS = SystemMessage(content=
"""You are a Writer.

You will receive:
- user_question
- plan (JSON)
- tool_results (JSON)

Rules:
- Write the final answer in Korean only (한국어로만 작성).
- Use tool_results as evidence. If you reference web search results, summarize them in Korean.
- Keep it concise and structured (bullets ok).
- If any tool output contains "Error" or "ERROR", explain briefly in Korean and propose a corrected plan.

Return ONLY JSON:
{"final": string}
""")


#### ㄴ 1.2.3 Planner Function

In [ ]:
import re, json

def extract_json(text: str) -> str:
    """LLM 응답에서 JSON만 뽑아낸다 (```json``` 코드블록/앞뒤 잡문 방어)."""
    if text is None:
        return ""
    t = text.strip()

    # 1) ```json ... ``` 블록 우선 추출
    m = re.search(r"```(?:json)?\s*(\{.*\})\s*```", t, flags=re.DOTALL | re.IGNORECASE)
    if m:
        return m.group(1).strip()

    # 2) 첫 { 부터 마지막 } 까지 잘라내기
    start = t.find("{")
    end = t.rfind("}")
    if start != -1 and end != -1 and end > start:
        return t[start:end+1].strip()

    return t


def make_plan(user_question: str) -> Plan:
    resp = llm.invoke([PLANNER_SYS, HumanMessage(content=user_question)])
    json_str = extract_json(resp.content)
    data = json.loads(resp.content)
    return Plan.model_validate(data)

#### ㄴ 1.2.4 Executor Function

In [ ]:
def _invoke_tool(alias: str, tool_input: str) -> Any:
    tool = TOOL_REGISTRY[alias]

    # TavilySearchResults는 보통 {"query": "..."} 형태가 안정적
    if alias == "search":
        try:
            return tool.invoke({"query": tool_input})
        except Exception:
            return tool.invoke(tool_input)

    # 나머지는 보통 문자열 입력
    try:
        return tool.invoke(tool_input)
    except Exception:
        return tool.run(tool_input)

def execute_plan(plan: Plan) -> List[Dict[str, Any]]:
    results = []
    for idx, step in enumerate(plan.steps):
        if step.tool == "final_answer":
            results.append({
                "step_index": idx,
                "tool": step.tool,
                "input": step.input,
                "output": "(final step - no tool execution)"
            })
            continue

        try:
            out = _invoke_tool(step.tool, step.input)
        except Exception as e:
            out = f"ERROR: {type(e).__name__}: {e}"

        results.append({
            "step_index": idx,
            "tool": step.tool,
            "input": step.input,
            "output": out
        })
    return results

#### ㄴ 1.2.5 Writer Function

In [ ]:
def write_final(user_question: str, plan: Plan, tool_results: List[Dict[str, Any]]) -> str:
    payload = {
        "user_question": user_question,
        "plan": plan.model_dump(),
        "tool_results": tool_results,
    }
    resp = llm.invoke([WRITER_SYS, HumanMessage(content=json.dumps(payload, ensure_ascii=False))])
    data = json.loads(resp.content)
    return data["final"]

#### ㄴ 1.2.6 User Prompt

In [ ]:
question = "Plan-Then-Execute 패턴이 뭐고 ReAct랑 차이를 예시 포함해서 설명해줘. 최신 관점이면 search도 활용해."

#### ㄴ 1.2.7 Result

In [ ]:
def run_plan_then_execute(question: str) -> str:
    # 1) Plan 생성
    plan = make_plan(question)

    # 2) 계획 실행 (결정론적)
    tool_results = execute_plan(plan)

    # 3) 최종 답변 생성
    final = write_final(question, plan, tool_results)

    return final

answer = run_plan_then_execute(question)
print(answer)

Plan-Then-Execute 패턴과 ReAct 패턴은 AI 에이전트 설계에서 자주 사용되는 두 가지 접근 방식입니다.

- **Plan-Then-Execute 패턴**:
  - **설명**: 이 패턴은 먼저 계획을 세운 후 실행하는 방식으로, 작업을 두 단계로 나눕니다. 계획 단계에서 목표를 설정하고 필요한 자원을 할당한 후, 실행 단계에서 계획을 수행합니다.
  - **특징**: 장기적인 목표 달성에 적합하며, 복잡한 작업이나 긴 시간 동안 지속되는 프로젝트에 유리합니다.
  - **예시**: 대규모 소프트웨어 개발 프로젝트에서 먼저 전체 아키텍처를 설계한 후, 각 모듈을 개발하는 방식.

- **ReAct 패턴**:
  - **설명**: ReAct는 Reasoning and Acting의 약자로, 실시간으로 상황을 분석하고 즉각적으로 반응하는 방식입니다.
  - **특징**: 빠른 의사결정과 즉각적인 반응이 필요한 상황에 적합하며, 단기적인 작업에 유리합니다.
  - **예시**: 고객 서비스 챗봇이 사용자 질문에 즉각적으로 답변하는 경우.

- **차이점**:
  - Plan-Then-Execute는 계획과 실행을 명확히 구분하여 체계적이고 장기적인 접근을 취하는 반면, ReAct는 실시간으로 상황에 대응하여 빠른 반응을 중시합니다.
  - Plan-Then-Execute는 복잡한 문제 해결에 강점을 가지며, ReAct는 신속한 문제 해결에 적합합니다.

이 두 패턴은 각각의 장단점이 있어, 상황에 따라 적절한 패턴을 선택하는 것이 중요합니다.


#### ㄴ 1.2.8 Debuging

In [ ]:
def debug_make_plan(user_question: str):
    resp = llm.invoke([PLANNER_SYS, HumanMessage(content=user_question)])
    print("=== RAW PLANNER OUTPUT ===")
    print(repr(resp.content))
    return resp.content

def debug_write_final(user_question: str, plan, tool_results):
    payload = {
        "user_question": user_question,
        "plan": plan.model_dump(),
        "tool_results": tool_results,
    }
    resp = llm.invoke([WRITER_SYS, HumanMessage(content=json.dumps(payload, ensure_ascii=False))])
    print("=== RAW WRITER OUTPUT ===")
    print(repr(resp.content))
    return resp.content


In [ ]:
question = "Plan-Then-Execute 패턴이 뭐고 ReAct랑 차이를 예시 포함해서 설명해줘. 최신 관점이면 search도 활용해."

plan = make_plan(question)
tool_results = execute_plan(plan)

raw_writer = debug_write_final(question, plan, tool_results)

KeyboardInterrupt: 

### ㄴ 1.3 Verbose

In [ ]:
def run_plan_then_execute_verbose(user_question: str, print_chars: int = 800):
    print("\n" + "="*80)
    print("USER QUESTION:")
    print(user_question)

    # PLAN 단계
    print("\n" + "-"*30 + " PLANNING " + "-"*30)

    plan = make_plan(user_question)

    print("GOAL:", plan.goal)
    print("\nSTEPS:")
    for i, step in enumerate(plan.steps):
        print(f"  [{i}] tool={step.tool} | input={step.input}")

    # EXECUTE 단계
    print("\n" + "-"*30 + " EXECUTION " + "-"*30)

    tool_results = execute_plan(plan)

    for r in tool_results:
        print(f"\n[STEP {r['step_index']}] TOOL: {r['tool']}")
        print("INPUT:", r["input"])

        out = r["output"]
        if not isinstance(out, str):
            out = json.dumps(out, ensure_ascii=False)

        print("OUTPUT (truncated):")
        print(out[:print_chars])

    # WRITE 단계
    print("\n" + "-"*30 + " WRITING FINAL ANSWER " + "-"*30)

    final = write_final(user_question, plan, tool_results)

    print("\nFINAL ANSWER:")
    print(final)

    print("\n" + "="*80)

    return {
        "plan": plan,
        "tool_results": tool_results,
        "final": final
    }


In [ ]:
run_plan_then_execute_verbose(question)

## ㄴ 2. State Machine (FSM-based Orchestration)

*    LLM은 **상태를 읽고 판단만** 함.
*    LLM은 이제 **행동 주체**가 아니라 **State**를 평가하는 **Decision Function**으로만 쓰임


***Plan-Then-Execute 코드는 그대로 재사용***

### ㄴ 2.1 Description

> **Plan-Then-Execute**는 **'무엇을 할지'** 정의한 패턴이고, <br>
**State Machine**은 **'그걸 어떻게 실행할지'** 를 정의하는 실행 모델이다.

**Plan-Then-Execute**와 **State Machine**의 차이
| Plan-Then-Execute       | State Machine                      |
| ------------ | -------------------------- |
| 함수가 흐름을 결정   | 상태가 흐름을 결정                 |
| 코드가 다음 단계 호출 | Transition Logic이 다음 상태 선택 |
| LLM = 작업 수행자 | LLM = 상태 판단자               |

<br>

**Plan-Then-Execute**에서의 역할이 **State Machine** 에서 아래와 같이 역할이 재배치됨

| 구성요소     | State Machine에서 역할     |
| -------- | ---------------------- |
| Planner  | `PLAN` 상태에서 호출되는 액션    |
| Executor | `EXECUTE` 상태에서 호출되는 액션 |
| Writer   | `WRITE` 상태에서 호출되는 액션   |




```javascript
TaskState:
    goal
    progress
    artifacts
    failures
```



### ㄴ 2.1 Code

#### ㄴ 2.1.0 Utils

##### ㄴ 2.1.0.1 Replan 전용 Planner 래퍼

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
import json

# (옵션) Replan 상황에서만 사용할 system prompt (기존 PLANNER_SYS를 안 건드리려면 별도 생성)
REPLAN_PLANNER_SYS = SystemMessage(content=
"""You are a Planner for an LLM agent system.

You MUST output ONLY valid JSON that matches this schema:
{
  "goal": string,
  "steps": [
    {"tool": "wiki|search|calculator|docstore_lookup|final_answer", "input": string}
  ]
}

Rules:
- Do NOT use markdown fences.
- If the user asks for an example, include a step that gathers info (wiki/search) AND ensure the final_answer step requests a concrete example with explicit state transitions.
- Prefer search over wiki when the term is ambiguous or not a Wikipedia entry (e.g., Plan-and-Execute in LLM agents).

Return ONLY JSON.
""")

# structured output으로 스키마 강제
replan_planner_llm = llm.with_structured_output(Plan)

def make_plan_replan(user_question: str, fix_hint: str | None = None) -> Plan:
    """
    Replan 전용 Planner.
    - fix_hint를 별도 채널로 주어 plan 스키마가 깨질 확률을 낮춤
    - structured_output(Plan)로 강제
    """
    payload = {
        "user_question": user_question,
        "fix_hint": fix_hint or ""
    }
    return replan_planner_llm.invoke([
        REPLAN_PLANNER_SYS,
        HumanMessage(content=json.dumps(payload, ensure_ascii=False))
    ])


##### ㄴ 2.1.0.2 State Machine 전용 Writer 래퍼

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
import json

STATE_WRITER_SYS = SystemMessage(content=
"""You are a Writer that produces the final answer for the user in Korean.

You receive a JSON payload with:
- user_question
- plan
- tool_results

Write a clear answer that MUST include:
1) 정의: Plan-Then-Execute와 State Machine 실행 모델의 관계
2) 차이: 단순 파이프라인(plan→execute→write) vs 상태기반 전이(phase-driven)
3) 구체 예시 1개 (연구/로봇/업무 자동화 중 아무거나)
4) 상태 전이 로그(한 줄): 순서대로 어떻게 되는지

Constraints:
- Keep it practical and concrete.
- Do not mention internal tool names.
- Output plain text (no JSON), Korean only.
""")

def write_final_state_machine(user_question: str, plan, tool_results) -> str:
    payload = {
        "user_question": user_question,
        "plan": plan.model_dump(),
        "tool_results": tool_results,
    }
    resp = llm.invoke([
        STATE_WRITER_SYS,
        HumanMessage(content=json.dumps(payload, ensure_ascii=False))
    ])
    return (resp.content or "").strip()


#### ㄴ 2.1.1 State Definition

In [ ]:
from typing import TypedDict, Literal, List, Dict, Any, Optional

Phase = Literal["PLAN", "EXECUTE", "VALIDATE", "WRITE", "DONE", "ERROR"]

class AgentState(TypedDict, total=False):
    # inputs
    user_question: str

    # state machine
    phase: Phase
    error: Optional[str]

    # artifacts (persisted across phases)
    plan: Any                 # Plan (pydantic model)
    tool_results: List[Dict[str, Any]]
    final: str

    # replan control
    replan_count: int
    max_replans: int

    fix_hint: str

    # debugging / tracing
    trace: List[Dict[str, Any]]

#### ㄴ 2.1.2 Validate Contract
> 지금까지 **실행한 결과**가 **다음 단계**로 넘어가도 될 만큼 **"신뢰 가능한 상태인가"** 를 판정

```javascript
EXECUTE → VALIDATE → (OK) → WRITE
                   → (FAIL) → REPLAN
```

###### ㄴ 2.1.2.1 Validate Schema

In [ ]:
from pydantic import BaseModel
from langchain_core.messages import SystemMessage, HumanMessage
import json

class ValidationOut(BaseModel):
    ok: bool
    reason: str
    fix_hint: str  # replan을 한다면 planner에게 줄 힌트(짧게)

###### ㄴ 2.1.2.2 Validate System Prompt

In [ ]:
VALIDATOR_SYS = SystemMessage(content=
"""You are a Validator for an LLM agent.

Input JSON contains:
- user_question
- plan
- tool_results

Decide if the information is sufficient to write a good answer.

Mark ok=false ONLY if:
- the retrieved content is clearly about the wrong concept (e.g., database query plan),
- or there is no material to produce at least one concrete example and a state-transition trace.

Otherwise ok=true.

Return ONLY JSON:
{"ok": boolean, "reason": string (Korean), "fix_hint": string (Korean)}
""")

#### ㄴ 2.1.3 FSM Execution Actions
> ***Plan-Then-Execute에서 사용한 함수 재사용***

##### ㄴ 2.1.3.1 Plan Action

In [ ]:
def action_plan(state: AgentState) -> AgentState:
    q = state["user_question"]
    fix_hint = (state.get("fix_hint") or "").strip()

    try:
        # replan일 때만 replan planner 사용
        if fix_hint:
            plan = make_plan_replan(q, fix_hint)
            used_replan_planner = True
        else:
            plan = make_plan(q)  # 기존 core planner
            used_replan_planner = False

        state["plan"] = plan
        state["trace"].append({
            "phase": "PLAN",
            "ok": True,
            "used_replan_planner": used_replan_planner,
            "steps": [s.model_dump() for s in plan.steps],
        })

        state["phase"] = "EXECUTE"
        state["fix_hint"] = ""  # 힌트 소모
        return state

    except Exception as e:
        state["error"] = f"PLAN_ERROR: {type(e).__name__}: {e}"
        state["trace"].append({"phase": "PLAN", "ok": False, "error": state["error"]})
        state["phase"] = "ERROR"
        return state


##### ㄴ 2.1.3.2 Execute Action

In [ ]:
def action_execute(state: AgentState) -> AgentState:
    try:
        plan = state["plan"]
        tool_results = execute_plan(plan)  # Plan-Then-Execute 에서의 Executer 함수
        state["tool_results"] = tool_results
        state["trace"].append({"phase":"EXECUTE", "ok": True, "num_results": len(tool_results)})
        state["phase"] = "VALIDATE"
    except Exception as e:
        state["error"] = f"EXECUTE_ERROR: {type(e).__name__}: {e}"
        state["trace"].append({"phase":"EXECUTE", "ok": False, "error": state["error"]})
        state["phase"] = "ERROR"
    return state

##### ㄴ 2.1.3.3 Write Action

In [ ]:
def action_write(state: AgentState) -> AgentState:
    try:
        q = state["user_question"]
        plan = state["plan"]
        tool_results = state.get("tool_results", [])
        final = write_final_state_machine(q, plan, tool_results)
        state["final"] = final
        state["trace"].append({"phase":"WRITE", "ok": True, "final_chars": len(final)})
        state["phase"] = "DONE"
    except Exception as e:
        state["error"] = f"WRITE_ERROR: {type(e).__name__}: {e}"
        state["trace"].append({"phase":"WRITE", "ok": False, "error": state["error"]})
        state["phase"] = "ERROR"
    return state

##### ㄴ 2.1.3.4 Validate Action

In [ ]:
import re

def action_validate(state: AgentState) -> AgentState:
    try:
        payload = {
            "user_question": state["user_question"],
            "plan": state["plan"].model_dump() if "plan" in state else None,
            "tool_results": state.get("tool_results", []),
        }
        # 구조화 출력(권장): JSON 깨짐 방지
        validator_llm = llm.with_structured_output(ValidationOut)

        verdict = validator_llm.invoke([
            VALIDATOR_SYS,
            HumanMessage(content=json.dumps(payload, ensure_ascii=False))
        ])

        state["trace"].append({
            "phase": "VALIDATE",
            "ok": True,
            "verdict": verdict.model_dump()
        })

        if verdict.ok:
            state["phase"] = "WRITE"
        else:
            # replan 제한
            state["replan_count"] = state.get("replan_count", 0) + 1
            if state["replan_count"] > state.get("max_replans", 2):
                state["error"] = f"VALIDATION_FAILED_MAX_REPLANS: {verdict.reason}"
                state["phase"] = "ERROR"
            else:
                # planner에게 힌트 남겨두기
                state["fix_hint"] = verdict.fix_hint
                state["trace"].append({
                    "phase": "REPLAN_TRIGGER",
                    "replan_count": state["replan_count"],
                    "reason": verdict.reason,
                    "fix_hint": verdict.fix_hint
                })
                # 힌트를 질문에 덧붙여 재계획 유도 (가장 단순한 방식)
                state["user_question"] = (
                    state["user_question"]
                    + "\n\n[Validator Hint]\n"
                    + verdict.fix_hint
                )
                state["phase"] = "PLAN"

    except Exception as e:
        state["error"] = f"VALIDATE_ERROR: {type(e).__name__}: {e}"
        state["trace"].append({"phase":"VALIDATE", "ok": False, "error": state["error"]})
        state["phase"] = "ERROR"

    return state


#### ㄴ 2.1.4 Transition Function (상태 전이 로직)

In [ ]:
def transition(state: AgentState) -> AgentState:
    phase = state["phase"]

    if phase == "PLAN":
        return action_plan(state)
    elif phase == "EXECUTE":
        return action_execute(state)
    elif phase == "VALIDATE":
        return action_validate(state)
    elif phase == "WRITE":
        return action_write(state)
    elif phase in ("DONE", "ERROR"):
        return state
    else:
        state["error"] = f"UNKNOWN_PHASE: {phase}"
        state["phase"] = "ERROR"
        return state

#### ㄴ 2.1.5 User Prompt

In [ ]:
question = "State Machine 실행 모델로 Plan-Then-Execute를 돌린다는 게 무슨 의미인지 예시로 설명해줘."

#### ㄴ 2.1.6 Result

In [ ]:
def run_state_machine(question: str) -> str:
    state: AgentState = {
        "user_question": question,
        "phase": "PLAN",
        "error": None,
        "trace": [],
        "replan_count": 0,
        "max_replans": 2,
        "fix_hint": "",
    }

    while state["phase"] not in ("DONE", "ERROR"):
        state = transition(state)

    if state["phase"] == "DONE":
        return state["final"]

    raise RuntimeError(state.get("error", "Unknown error"))

In [ ]:
print(run_state_machine(question))

1) **정의**: Plan-Then-Execute 모델은 계획을 세운 후 이를 실행하는 두 단계로 구성된 접근 방식입니다. 이 모델을 상태 기계(State Machine)와 결합하면, 계획 단계에서 설정된 초기 상태를 기반으로 상태 기계가 실행 단계에서 다양한 상태 전이를 통해 작업을 수행합니다. 상태 기계는 외부 이벤트에 따라 상태를 전이하며, 각 상태에서 수행할 작업을 결정합니다.

2) **차이**: 단순 파이프라인 방식에서는 계획(plan) 단계에서 실행(execute) 단계로 넘어가고, 그 결과를 작성(write)하는 일련의 고정된 절차를 따릅니다. 반면, 상태기반 전이 방식에서는 각 단계가 상태에 따라 유동적으로 전이됩니다. 즉, 상태 기계는 각 상태에서 발생하는 이벤트에 따라 다음 상태로 전이하며, 이 과정에서 필요한 작업을 수행합니다. 이는 복잡한 작업을 보다 유연하게 처리할 수 있게 해줍니다.

3) **구체 예시**: 로봇 청소기의 경우를 생각해봅시다. 로봇 청소기는 방 청소 계획을 세운 후, 상태 기계를 통해 이를 실행합니다. 초기 상태는 '대기' 상태로 시작하며, '청소 시작' 이벤트가 발생하면 '청소 중' 상태로 전이됩니다. 청소 중에 장애물을 만나면 '회피' 상태로 전이하여 장애물을 피하고, 다시 '청소 중' 상태로 돌아갑니다. 모든 청소가 완료되면 '완료' 상태로 전이하여 작업을 종료합니다.

4) **상태 전이 로그**: 
   - 대기 → 청소 시작 → 청소 중 → 장애물 회피 → 청소 중 → 완료

이와 같이 상태 기계는 각 상태에서의 이벤트에 따라 유연하게 전이하며, 복잡한 작업을 효율적으로 수행할 수 있습니다.


### ㄴ 2.2 Verbose

In [ ]:
import json

def run_state_machine_verbose(question: str, print_chars: int = 600, max_steps: int = 20) -> AgentState:
    state: AgentState = {
        "user_question": question,
        "phase": "PLAN",
        "error": None,
        "trace": [],
        "replan_count": 0,
        "max_replans": 2,
        "fix_hint": "",
    }

    print("\n" + "="*80)
    print("USER QUESTION:")
    print(question)

    steps = 0
    while state["phase"] not in ("DONE", "ERROR"):
        steps += 1
        if steps > max_steps:
            state["error"] = "MAX_STEPS_EXCEEDED"
            state["phase"] = "ERROR"
            break

        print("\n" + "-"*28 + f" PHASE: {state['phase']} " + "-"*28)
        state = transition(state)

        if state["phase"] == "EXECUTE" and "plan" in state:
            print("Plan ready. Steps:")
            for i, s in enumerate(state["plan"].steps):
                print(f"  [{i}] {s.tool} :: {s.input}")

        if state["phase"] == "VALIDATE":
            print(f"Executed {len(state.get('tool_results', []))} steps. (Next: VALIDATE)")

        if state["phase"] == "PLAN" and state.get("replan_count", 0) > 0:
            print(f"Replanning... (count={state['replan_count']}/{state['max_replans']})")

        if state["phase"] == "WRITE":
            print("Validation passed. Moving to WRITE.")

    print("\n" + "-"*28 + f" PHASE: {state['phase']} " + "-"*28)

    if state["phase"] == "DONE":
        print("\nFINAL:")
        print(state["final"])
    else:
        print("\nERROR:")
        print(state.get("error"))

    print("\nTRACE (last 6):")
    trace = state.get("trace", [])
    print(json.dumps(trace[-6:], ensure_ascii=False, indent=2))
    print("="*80)

    return state

In [ ]:
run_state_machine_verbose(question)


USER QUESTION:
State Machine 실행 모델로 Plan-Then-Execute를 돌린다는 게 무슨 의미인지 예시로 설명해줘.

---------------------------- PHASE: PLAN ----------------------------
Plan ready. Steps:
  [0] wiki :: State machine
  [1] wiki :: Plan-Then-Execute model
  [2] final_answer :: A State Machine execution model involves defining a set of states and transitions between those states based on inputs or events. In a Plan-Then-Execute model, a plan is created first, detailing the sequence of actions or states needed to achieve a goal, and then this plan is executed step-by-step. For example, consider a robot vacuum cleaner: the planning phase involves mapping out the room and determining the optimal path to clean it, while the execution phase involves the vacuum following this path, transitioning between states like 'moving', 'cleaning', and 'charging' based on its current position and battery level.

---------------------------- PHASE: EXECUTE ----------------------------
Executed 3 steps. (Next: VALIDATE)

---

{'user_question': 'State Machine 실행 모델로 Plan-Then-Execute를 돌린다는 게 무슨 의미인지 예시로 설명해줘.\n\n[Validator Hint]\nPlan-Then-Execute 모델에 대한 올바른 설명을 찾기 위해 추가적인 자료 조사를 수행하세요. 예를 들어, 로봇 공학이나 인공지능에서의 Plan-Then-Execute 모델에 대한 설명을 찾아보세요.',
 'phase': 'DONE',
 'error': None,
 'trace': [{'phase': 'PLAN',
   'ok': True,
   'used_replan_planner': False,
   'steps': [{'tool': 'wiki', 'input': 'State machine'},
    {'tool': 'wiki', 'input': 'Plan-Then-Execute model'},
    {'tool': 'final_answer',
     'input': "A State Machine execution model involves defining a set of states and transitions between those states based on inputs or events. In a Plan-Then-Execute model, a plan is created first, detailing the sequence of actions or states needed to achieve a goal, and then this plan is executed step-by-step. For example, consider a robot vacuum cleaner: the planning phase involves mapping out the room and determining the optimal path to clean it, while the execution phase involves the vacuum following this path

## ㄴ 3. Graph Execution (DAG Workflow Engine)
> Agent는 더 이상 **Loop**가 아닌 **Directed Graph** **(tool orchestration)**

### ㄴ 3.1 Description

- State Machine은 **"어떻게 상태가 바뀌는가?"** 를 정의 (흐름을 관리하는 방식)
- Graph Execution은 **"작업들이 어떤 의존관계로 연결되는가?"** 를 정의 (작업 구조를 표현하는 방식)

- **Agent Framework Structure**
| Framework             | internal structure
| ----------------- | ---------------------------------------- |
| LangGraph         | Graph + State control                    |
| AutoGen           | Conversation State Machine + Tool Graph  |
| CrewAI            | Role-based State + Task Graph            |
| OpenAI Assistants | Run State Machine + Tool Execution Graph |

- 구조 철학 :
  - Execution = Graph
  - Control = State Machine




### ㄴ 3.2 Code
> State Machine의 **while-loop** 를 LangGraph의 **StateGraph(Node/Edge)** 로 실행 흐름으로 변경

#### ㄴ 3.2.0 utils

##### ㄴ 3.2.0.1 JSON Parser

In [ ]:
def _strip_json_fence(text: str) -> str:
    t = text.strip()
    # ```json ... ``` 제거
    if t.startswith("```"):
        t = re.sub(r"^```[a-zA-Z0-9]*\n", "", t)
        t = re.sub(r"\n```$", "", t)
    return t.strip()

def parse_plan(text: str) -> Plan:
    t = _strip_json_fence(text)
    obj = json.loads(t)
    return Plan.model_validate(obj)


#### ㄴ 3.2.1 Contract

##### ㄴ 3.2.1.1 StateGraph Schema and Type Definition

In [ ]:
import json
import re
from typing import TypedDict, List, Dict, Any, Optional

from pydantic import BaseModel, Field, ValidationError
from langchain_core.messages import SystemMessage, HumanMessage

class Step(BaseModel):
    tool: str
    input: str

class Plan(BaseModel):
    goal: str
    steps: List[Step]

class GraphState(TypedDict, total=False):
    user_question: str

    plan: Plan
    raw_plan_text: str

    tool_results: List[Dict[str, Any]]

    validation_ok: bool
    validation_reason: str
    fix_hint: str

    replan_count: int
    max_replans: int

    final: str

##### ㄴ 3.2.1.2 System Prompt

###### ㄴ 3.2.1.2.1 Planner System Prompt

In [ ]:
PLANNER_SYS = SystemMessage(content=
"""You are a Planner for an LLM agent system.

Output ONLY valid JSON matching this schema:
{
  "goal": string,
  "steps": [
    {"tool": "wiki|search|calculator|docstore_lookup|final_answer", "input": string}
  ]
}

Rules:
- Do NOT use markdown fences.
- Prefer search over wiki if the term is ambiguous (e.g., Plan-and-Execute in LLM agents).
- Ensure the last step is always tool=final_answer.

Return ONLY JSON.
""")

###### ㄴ 3.2.1.2.2 Replan Planner System Prompt

In [ ]:
REPLAN_PLANNER_SYS = SystemMessage(content=
"""You are a Replanning Planner.

You are given:
- user question
- validator fix_hint (what was wrong and what to fix)

Output ONLY valid JSON matching this schema:
{
  "goal": string,
  "steps": [
    {"tool": "wiki|search|calculator|docstore_lookup|final_answer", "input": string}
  ]
}

Rules:
- Do NOT use markdown fences.
- Fix the issue described by fix_hint.
- Prefer search if Wikipedia is likely to be misleading/ambiguous.
- Ensure the last step is always tool=final_answer.

Return ONLY JSON.
""")

###### ㄴ 3.2.1.2.3 Validator System Prompt

In [ ]:
VALIDATOR_SYS = SystemMessage(content=
"""You are a Validator.

Input: JSON with keys:
- user_question
- plan
- tool_results

Decide whether the gathered information is sufficient and on-topic.

Return ONLY JSON:
{
  "ok": boolean,
  "reason": string (Korean),
  "fix_hint": string (Korean, concrete instruction for replanning)
}

Mark ok=false if:
- the info is about the wrong concept, OR
- the answer would be too vague (missing concrete example / missing relationship).

Return ONLY JSON.
""")

###### ㄴ 3.2.1.2.4 Writer System Prompt

In [ ]:
GRAPH_WRITER_SYS = SystemMessage(content=
"""You are a Writer that produces the final answer for the user in Korean.

You receive a JSON payload with:
- user_question
- plan
- tool_results

Write a clear answer that MUST include:
1) 정의: Plan-Then-Execute와 Graph/State-based 실행의 관계
2) 예시 1개 (로봇/업무 자동화/리서치 중 택1)
3) (선택) 실패 시 재계획이 왜 필요한지 한 문장

Constraints:
- Keep it practical and concrete.
- Do not mention internal tool names.
- Output plain Korean text only.
""")


#### ㄴ 3.2.2 Fucntions

##### ㄴ 3.2.2.1. Planner Function

In [ ]:
def make_plan(user_question: str) -> Plan:
    resp = llm.invoke([PLANNER_SYS, HumanMessage(content=user_question)])
    raw = resp.content or ""
    return parse_plan(raw)

##### ㄴ 3.2.2.2. Replan Planner Function

In [ ]:
def make_plan_replan(user_question: str, fix_hint: str) -> Plan:
    payload = {"user_question": user_question, "fix_hint": fix_hint}
    resp = llm.invoke([REPLAN_PLANNER_SYS, HumanMessage(content=json.dumps(payload, ensure_ascii=False))])
    raw = resp.content or ""
    return parse_plan(raw)

##### ㄴ 3.2.2.3. Validator Function

In [ ]:
def validate_results(user_question: str, plan: Plan, tool_results: List[Dict[str, Any]]) -> Dict[str, Any]:
    payload = {
        "user_question": user_question,
        "plan": plan.model_dump(),
        "tool_results": tool_results,
    }
    resp = llm.invoke([VALIDATOR_SYS, HumanMessage(content=json.dumps(payload, ensure_ascii=False))])
    t = _strip_json_fence(resp.content or "")
    return json.loads(t)

##### ㄴ 3.2.2.1. Writer Function

In [ ]:
def write_final_graph(user_question: str, plan: Plan, tool_results: List[Dict[str, Any]]) -> str:
    payload = {
        "user_question": user_question,
        "plan": plan.model_dump(),
        "tool_results": tool_results,
    }
    resp = llm.invoke([GRAPH_WRITER_SYS, HumanMessage(content=json.dumps(payload, ensure_ascii=False))])
    return (resp.content or "").strip()

NameError: name 'Plan' is not defined

#### ㄴ 3.2.3. Routing Tools

In [ ]:
tool_map = {
    "wiki": wiki,                 # WikipediaQueryRun
    "search": search_tool,        # TavilySearchResults
    "calculator": calculator,     # tool
    "docstore_lookup": docstore_lookup,  # tool
}

def run_tool(tool_name: str, tool_input: str) -> Any:
    if tool_name not in tool_map:
        raise ValueError(f"Unknown tool: {tool_name}")
    return tool_map[tool_name].invoke(tool_input)

#### ㄴ 3.2.4. Plan Executor

In [ ]:
def execute_plan(plan: Plan) -> List[Dict[str, Any]]:
    results: List[Dict[str, Any]] = []
    for i, step in enumerate(plan.steps):
        if step.tool == "final_answer":
            results.append({
                "step_index": i,
                "tool": "final_answer",
                "input": step.input,
                "output": "(final step - no tool execution)"
            })
            continue

        out = run_tool(step.tool, step.input)
        results.append({
            "step_index": i,
            "tool": step.tool,
            "input": step.input,
            "output": out,
        })
    return results

#### ㄴ 3.2.5. Nodes

In [ ]:
from langgraph.graph import StateGraph, END

##### ㄴ 3.2.5.1 Planner Node

In [ ]:
def node_plan(state: GraphState) -> GraphState:
    q = state["user_question"]
    plan = make_plan(q)
    return {
        **state,
        "plan": plan,
        "tool_results": [],
    }

##### ㄴ 3.2.5.2 Executor Node

In [ ]:
def node_execute(state: GraphState) -> GraphState:
    plan = state["plan"]
    tool_results = execute_plan(plan)
    return {**state, "tool_results": tool_results}

##### ㄴ 3.2.5.3 Validator Node

In [ ]:
def node_validate(state: GraphState) -> GraphState:
    q = state["user_question"]
    plan = state["plan"]
    tool_results = state.get("tool_results", [])
    verdict = validate_results(q, plan, tool_results)
    return {
        **state,
        "validation_ok": bool(verdict.get("ok", False)),
        "validation_reason": verdict.get("reason", ""),
        "fix_hint": verdict.get("fix_hint", ""),
    }

##### ㄴ 3.2.5.4 Replanner Node

In [ ]:
def node_replan(state: GraphState) -> GraphState:
    # replan_count 증가
    cnt = int(state.get("replan_count", 0)) + 1
    q = state["user_question"]
    fix_hint = state.get("fix_hint", "").strip()

    plan = make_plan_replan(q, fix_hint)
    return {
        **state,
        "replan_count": cnt,
        "plan": plan,
        "tool_results": [],
    }

##### ㄴ 3.2.5.5 Writer Node

In [ ]:
def node_write(state: GraphState) -> GraphState:
    q = state["user_question"]
    plan = state["plan"]
    tool_results = state.get("tool_results", [])
    final = write_final_graph(q, plan, tool_results)
    return {**state, "final": final}

#### ㄴ 3.2.6 Condition function (Transition Function)
> ***다음 노드 이름을 반환***

- **Graph Execution**은 **Sate Machine** 과 다르게 상태를 변경하는 게 아니라
**DAG 위에서 다음 Node를 선택**해야 하기 때문

In [ ]:
def route_after_validate(state: GraphState) -> str:
    ok = bool(state.get("validation_ok", False))
    if ok:
        return "WRITE"

    # fail이면 replan_count 확인
    cnt = int(state.get("replan_count", 0))
    max_r = int(state.get("max_replans", 2))
    if cnt >= max_r:
        return "END_FAIL"
    return "REPLAN"

#### ㄴ 3.2.7 Edges

In [ ]:
graph = StateGraph(GraphState)

graph.add_node("PLAN", node_plan)
graph.add_node("EXECUTE", node_execute)
graph.add_node("VALIDATE", node_validate)
graph.add_node("REPLAN", node_replan)
graph.add_node("WRITE", node_write)

graph.set_entry_point("PLAN")
graph.add_edge("PLAN", "EXECUTE")
graph.add_edge("EXECUTE", "VALIDATE")

graph.add_conditional_edges(
    "VALIDATE",
    route_after_validate,
    {
        "WRITE": "WRITE",
        "REPLAN": "REPLAN",
        "END_FAIL": END,
    }
)

graph.add_edge("REPLAN", "EXECUTE")
graph.add_edge("WRITE", END)

app = graph.compile()

#### ㄴ 3.2.7 User Prompt

In [ ]:
question = "Graph 실행(노드/엣지) 모델로 Plan-Then-Execute를 돌린다는 게 무슨 의미인지, 예시로 설명해줘."

#### ㄴ 3.2.8 Result

In [ ]:
def run_graph(question: str, max_replans: int = 2) -> str:
    init: GraphState = {
        "user_question": question,
        "replan_count": 0,
        "max_replans": max_replans,
    }
    out = app.invoke(init)
    if "final" in out and out["final"]:
        return out["final"]
    # 실패 케이스
    raise RuntimeError(f"GRAPH_FAILED: validation_reason={out.get('validation_reason')}")

In [ ]:
print(run_graph(question))

#### ㄴ 3.2.9 Verbose

In [ ]:
def run_graph_verbose(question: str, max_replans: int = 2) -> Dict[str, Any]:
    init: GraphState = {
        "user_question": question,
        "replan_count": 0,
        "max_replans": max_replans,
    }

    print("\n" + "="*80)
    print("USER QUESTION:")
    print(question)

    last = None
    for event in app.stream(init, stream_mode="values"):
        last = event
        # 어떤 노드까지 갔는지 추적하기 쉽게 주요 필드만 출력
        print("\n--- STATE UPDATE ---")
        keys = ["plan", "tool_results", "validation_ok", "validation_reason", "fix_hint", "replan_count", "final"]
        for k in keys:
            if k in event:
                if k == "plan":
                    p = event["plan"]
                    print("plan.goal:", p.goal)
                    print("plan.steps:", [f"{s.tool}::{s.input}" for s in p.steps])
                elif k == "tool_results":
                    print("tool_results.count:", len(event["tool_results"]))
                    if event["tool_results"]:
                        sample = event["tool_results"][-1]
                        print("last_tool:", sample.get("tool"))
                else:
                    print(f"{k}:", event[k])

    print("\n" + "="*80)
    if last and "final" in last:
        print("FINAL:\n", last["final"])
    return last

In [ ]:
_ = run_graph_verbose(question)

## ㄴ 4. DAG-Orchestrated Stateful Execution (FSM + Graph)
> **Planner**  → 전체 전략 생성 (FSM의 PLAN 역할)</br>
**Executor** → 실제 작업 수행 (Graph/DAG 실행)


### ㄴ 4.1 Description

- FSM이 **시스템 제어**
- Graph가 **실제 작업 실행**
- LLM은 **역할별로만 호출**됨

#### ㄴ 4.1.1 Architecture

- FSM
```text
┌------┐
│ PLAN │  LLM(Planner) → Plan(DAG spec)
└--┬--┘
    ▼
┌--------┐
│EXECUTE │  Graph Engine runs DAG nodes (tools/mini-LLMs) possibly in parallel
└--┬----┘
    ▼
┌-----------------┐
│     VALIDATE    │  LLM(Validator) checks quality/safety/coverage
└┬------------┬-┘
  │OK          │FAIL
  ▼            ▼
┌------┐   ┌-------┐
│ WRITE│   │ REPLAN│  LLM(Replan Planner) adjusts plan with fix hints
└-┬---┘   └-┬----┘
   ▼           └----- back to EXECUTE (or PLAN)
  DONE
```

- DAG
```javascript
search(A) ┐
wiki(A)   ├─► merge_A ┐
          │            │
search(B) ┤            ├──► compare ─► write_inputs
wiki(B)   ┘            │
                        └──► citations_pack
```

- FSM은 **"상태/재시도/검증/가드레일"** 만 담당
- EXECUTE는 **LangGraph DAG를 실행하는 함수** 로 캡슐화

### ㄴ 4.2 Code

#### ㄴ 4.2.0 utils

##### ㄴ 4.2.0.1 JSON Parser

In [ ]:
import json, re
from typing import Any, Dict, List, Literal, TypedDict

def strip_json_fences(text: str) -> str:
    text = text.strip()
    m = re.match(r"^```(?:json)?\s*(.*?)\s*```$", text, flags=re.DOTALL | re.IGNORECASE)
    return m.group(1).strip() if m else text

def safe_load_json(text: str) -> Dict[str, Any]:
    return json.loads(strip_json_fences(text))

#### ㄴ 4.2.1 Schema

In [ ]:
from pydantic import BaseModel, Field

ToolName = Literal["wiki", "search", "calculator", "docstore_lookup", "final_answer"]

class Step(BaseModel):
    tool: ToolName
    input: str

class Plan(BaseModel):
    goal: str
    steps: List[Step] = Field(default_factory=list)

class Verdict(BaseModel):
    ok: bool
    reason: str
    fix_hint: str = ""

#### ㄴ 4.2.2 DAG (Data Plane)

In [ ]:
from typing import TypedDict, List, Dict, Any

class ExecState(TypedDict, total=False):
    # 입력
    user_question: str
    plan: Any

    # lane별 중간 산출물
    A_search: Any
    A_wiki: Any
    B_search: Any
    B_wiki: Any

    # join 결과
    merged_A: str
    merged_B: str
    comparison: str

    # 최종 tool_results로 합치기(기존 파이프라인 호환)
    tool_results: List[Dict[str, Any]]


def run_tool(tool: str, inp: str) -> str:
    if tool == "wiki":
        return wiki.run(inp)
    if tool == "search":
        return json.dumps(search_tool.invoke({"query": inp}), ensure_ascii=False)
    if tool == "calculator":
        return calculator(inp)
    if tool == "docstore_lookup":
        return docstore_lookup(inp)
    if tool == "final_answer":
        return "(final step - no tool execution)"
    raise ValueError(f"Unknown tool: {tool}")

def node_execute_steps(state: ExecState) -> Dict[str, Any]:
    plan_obj = Plan.model_validate(state["plan"])
    results = []
    for i, step in enumerate(plan_obj.steps):
        out = run_tool(step.tool, step.input)
        results.append({
            "step_index": i,
            "tool": step.tool,
            "input": step.input,
            "output": out
        })
    return {"tool_results": results}

##### ㄴ 4.2.2.2 Nodes

In [ ]:
def node_search_A(state: ExecState) -> ExecState:
    q = state["user_question"]
    out = search_tool.invoke({"query": f"{q} A 관점"})  # Tavily 형식에 맞게
    return {"A_search": out}

def node_wiki_A(state: ExecState) -> ExecState:
    q = state["user_question"]
    out = wiki.invoke(f"{q} A 키워드")
    return {"A_wiki": out}

def node_search_B(state: ExecState) -> ExecState:
    q = state["user_question"]
    out = search_tool.invoke({"query": f"{q} B 관점"})
    return {"B_search": out}

def node_wiki_B(state: ExecState) -> ExecState:
    q = state["user_question"]
    out = wiki.invoke(f"{q} B 키워드")
    return {"B_wiki": out}


def node_merge_A(state: ExecState) -> ExecState:
    # lane A 결과를 짧게 합치기
    s = state.get("A_search")
    w = state.get("A_wiki")
    merged = f"[A]\n- search: {str(s)[:400]}\n- wiki: {str(w)[:400]}"
    return {"merged_A": merged}

def node_merge_B(state: ExecState) -> ExecState:
    s = state.get("B_search")
    w = state.get("B_wiki")
    merged = f"[B]\n- search: {str(s)[:400]}\n- wiki: {str(w)[:400]}"
    return {"merged_B": merged}

def node_compare(state: ExecState) -> ExecState:
    a = state.get("merged_A", "")
    b = state.get("merged_B", "")
    comparison = f"COMPARE:\n{a}\n\n{b}"

    tool_results = [
        {"tool": "search(A)", "input": "A 관점", "output": state.get("A_search")},
        {"tool": "wiki(A)", "input": "A 키워드", "output": state.get("A_wiki")},
        {"tool": "search(B)", "input": "B 관점", "output": state.get("B_search")},
        {"tool": "wiki(B)", "input": "B 키워드", "output": state.get("B_wiki")},
        {"tool": "compare", "input": "merge+compare", "output": comparison},
    ]
    return {"comparison": comparison, "tool_results": tool_results}


In [ ]:
from langgraph.graph import StateGraph, END

exec_builder = StateGraph(ExecState)

# fan-out nodes
exec_builder.add_node("node_search_A", node_search_A)
exec_builder.add_node("node_wiki_A", node_wiki_A)
exec_builder.add_node("node_search_B", node_search_B)
exec_builder.add_node("node_wiki_B", node_wiki_B)

# join nodes
exec_builder.add_node("node_merge_A", node_merge_A)
exec_builder.add_node("node_merge_B", node_merge_B)
exec_builder.add_node("node_compare", node_compare)

# entry
exec_builder.set_entry_point("node_search_A")

# fan-out 구조(여기선 단순히 분기 엣지로 표현)
exec_builder.add_edge("node_search_A", "node_wiki_A")
exec_builder.add_edge("node_search_A", "node_search_B")   # 가지치기
exec_builder.add_edge("node_search_A", "node_wiki_B")     # 가지치기

# A lane join
exec_builder.add_edge("node_wiki_A", "node_merge_A")

# B lane join
exec_builder.add_edge("node_search_B", "node_merge_B")
exec_builder.add_edge("node_wiki_B", "node_merge_B")

# final join
exec_builder.add_edge("node_merge_A", "node_compare")
exec_builder.add_edge("node_merge_B", "node_compare")

exec_builder.add_edge("node_compare", END)

exec_graph = exec_builder.compile()

#### ㄴ 4.2.3 FSM(Control Plane)

##### ㄴ 4.2.3.1 Contract

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

###### ㄴ 4.2.3.1.1 Planner System Prompt

In [ ]:
PLANNER_SYS = SystemMessage(content="""
You are the PLANNER.
Return JSON ONLY matching:
{"goal": "...", "steps":[{"tool":"...","input":"..."}...]}

Rules:
- tools allowed: wiki, search, calculator, docstore_lookup, final_answer
- prefer search for modern LLM-agent concepts; wiki can be ambiguous
- steps <= 5
- last step MUST be {"tool":"final_answer","input":"..."}
""")

###### ㄴ 4.2.3.1.2 RePlanner System Prompt

In [ ]:
REPLANNER_SYS = SystemMessage(content="""
You are the REPLANNER.
Input: JSON {"user_question": "...", "fix_hint": "..."}.
Return a revised plan JSON in the same schema.
Rules:
- follow fix_hint precisely
- steps <= 5, last step final_answer
""")

###### ㄴ 4.2.3.1.3 Validator System Prompt

In [ ]:
# FIX : Miss Direction
VALIDATOR_SYS = SystemMessage(content="""
You are the VALIDATOR for an agent pipeline (artifacts-only).

Input JSON:
{
  "user_question": "...",
  "plan": {...},
  "tool_results": [...]
}

You must NOT evaluate any drafted/final answer text.
Your job is to decide whether the collected artifacts are sufficient to write a good answer.

Return JSON ONLY:
{"ok": true/false, "reason":"...", "fix_hint":"..."}

Rules:
- ok=true if the artifacts contain enough material to answer the question well.
- ok=false ONLY when missing required evidence/material.

What to check (practical):
1) COVERAGE: Do artifacts cover the key sub-questions implied by user_question?
2) EXAMPLES: If the user asked for examples ("예시 포함"), do artifacts include at least one concrete example or case?
3) DISAMBIGUATION: If there's a known confusion risk (e.g., Plan-Then-Execute vs DB query plan),
   do artifacts contain a correct definition/source that resolves it?

Do NOT require citations or stylistic constraints.
Do NOT demand specific metaphors/phrases.
Do NOT mention tools or internal node names.

When ok=false:
- reason: short label + 1 sentence (e.g., "MISSING_EXAMPLE: ...")
- fix_hint: MUST be actionable and MUST start with one of:
  - "Search for ..."
  - "Add a node to fetch ..."
  - "Add a node to summarize ..."
  - "Remove unrelated artifact ..."

Important:
- Even when ok=true, you MUST still include non-empty "reason" and "fix_hint" (for schema compatibility).
  Use:
  reason="OK"
  fix_hint=""

""")

###### ㄴ 4.2.3.1.4 Writer System Prompt

In [ ]:
WRITER_SYS = SystemMessage(content="""
You are the WRITER. Output Korean only (plain text).
Input JSON {"user_question":..., "plan":..., "tool_results":...}
Write a practical, concrete answer. Do not mention internal tool names.
""")

##### ㄴ 4.2.3.3 FSM State Definiton & Execution Actions



###### ㄴ 4.2.3.3.0 Normalize Utils

In [ ]:
from typing import Tuple

def normalize_plan_dict(raw: Any, user_question: str) -> Dict[str, Any]:
    """
    Convert various LLM outputs into valid Plan dict:
    { "goal": str, "steps": [ {"tool": "...", "input": "..."} ] }
    """
    if not isinstance(raw, dict):
        # raw가 그냥 문자열이면 -> 최소 플랜
        return {
            "goal": user_question,
            "steps": [
                {"tool": "search", "input": user_question},
                {"tool": "final_answer", "input": "도구 결과를 바탕으로 한국어로 답변해줘."},
            ],
        }

    goal = raw.get("goal") or raw.get("objective") or raw.get("task") or user_question
    steps = raw.get("steps", [])

    # steps가 문자열 리스트인 경우: heuristic로 tool 추정
    if isinstance(steps, list) and (len(steps) > 0) and isinstance(steps[0], str):
        new_steps = []
        for s in steps:
            s = str(s).strip()
            # "final_answer: ...." 같은 형식 지원
            m = re.match(r"^(wiki|search|calculator|docstore_lookup|final_answer)\s*:\s*(.*)$", s, re.I)
            if m:
                tool = m.group(1).lower()
                inp = m.group(2).strip()
            else:
                # 문장에 Search/wiki 같은 키워드가 있으면 search/wiki로
                low = s.lower()
                if "wiki" in low or "wikipedia" in low:
                    tool, inp = "wiki", s
                elif "calc" in low or "calculate" in low or re.search(r"[\d\)\(]\s*[\+\-\*/]", s):
                    tool, inp = "calculator", s
                else:
                    tool, inp = "search", s
            new_steps.append({"tool": tool, "input": inp})

        # 마지막은 final_answer 강제
        if not new_steps or new_steps[-1]["tool"] != "final_answer":
            new_steps.append({"tool": "final_answer", "input": "도구 결과를 바탕으로 한국어로 최종 답변을 작성해줘."})

        return {"goal": goal, "steps": new_steps}

    # steps가 dict 리스트인데 tool/input 키가 다를 때 보정
    if isinstance(steps, list) and (len(steps) > 0) and isinstance(steps[0], dict):
        new_steps = []
        for step in steps:
            tool = step.get("tool") or step.get("action") or step.get("name")
            inp  = step.get("input") or step.get("query") or step.get("args") or step.get("text")
            if tool is None or inp is None:
                # 누락되면 search로 강제
                new_steps.append({"tool": "search", "input": json.dumps(step, ensure_ascii=False)})
            else:
                tool = str(tool).strip()
                inp = str(inp).strip()
                # tool 정규화
                tool_map = {
                    "tavily": "search",
                    "tavily_search": "search",
                    "wikipedia": "wiki",
                    "calc": "calculator",
                }
                tool = tool_map.get(tool.lower(), tool.lower())
                if tool not in {"wiki","search","calculator","docstore_lookup","final_answer"}:
                    tool = "search"
                new_steps.append({"tool": tool, "input": inp})

        if not new_steps or new_steps[-1]["tool"] != "final_answer":
            new_steps.append({"tool": "final_answer", "input": "도구 결과를 바탕으로 한국어로 최종 답변을 작성해줘."})

        return {"goal": goal, "steps": new_steps}

    # steps가 이상하면 최소 플랜
    return {
        "goal": goal,
        "steps": [
            {"tool": "search", "input": user_question},
            {"tool": "final_answer", "input": "도구 결과를 바탕으로 한국어로 답변해줘."},
        ],
    }

def make_plan_safe(user_question: str, planner_sys: SystemMessage = None) -> Plan:
    sys_msg = planner_sys or PLANNER_SYS
    resp = llm.invoke([sys_msg, HumanMessage(content=user_question)])

    try:
        raw = safe_load_json(resp.content)
    except Exception:
        raw = {"goal": user_question, "steps": [resp.content]}

    normalized = normalize_plan_dict(raw, user_question)
    return Plan.model_validate(normalized)


###### ㄴ 4.2.3.3.1 State Definition

In [ ]:
class AgentState(TypedDict):
    user_question: str
    phase: str
    plan: Dict[str, Any]
    tool_results: List[Dict[str, Any]]
    verdict: Dict[str, Any]
    fix_hint: str
    replan_count: int
    max_replans: int
    draft_answer: str
    final: str
    trace: List[Dict[str, Any]]

###### ㄴ 4.2.3.3.2 Planner Action

In [ ]:
def fsm_plan(state: AgentState) -> AgentState:
    try:
        plan_obj = make_plan_safe(state["user_question"], PLANNER_SYS)
        state["plan"] = plan_obj.model_dump()
        state["phase"] = "EXECUTE"
        state["trace"].append({"phase":"PLAN","ok":True,"steps":state["plan"]["steps"]})
        return state
    except Exception as e:
        state["phase"] = "DONE"
        state["final"] = f"PLAN_ERROR: {type(e).__name__}: {e}"
        state["trace"].append({"phase":"PLAN","ok":False,"error":state["final"]})
        return state

###### ㄴ 4.2.3.3.3 Executor Action

In [ ]:
def fsm_execute(state: AgentState) -> AgentState:
    executed = []

    gstate: ExecState = {
        "user_question": state["user_question"],
        "plan": state["plan"],
        "tool_results": [],
    }

    for ev in exec_graph.stream(gstate, stream_mode="updates"):
        for node_name, upd in ev.items():
            executed.append(node_name)
            if isinstance(upd, dict) and "tool_results" in upd:
                gstate["tool_results"] = upd["tool_results"]

    state["tool_results"] = gstate.get("tool_results", [])
    state["trace"].append({"phase": "EXECUTE", "ok": True, "executed_nodes": executed})

    # FIX : Cost 증가 이슈 있음
    payload = {
        "user_question": state["user_question"],
        "plan": state["plan"],
        "tool_results": state["tool_results"],
    }
    draft = llm.invoke([WRITER_SYS, HumanMessage(content=json.dumps(payload, ensure_ascii=False))]).content
    state["draft_answer"] = draft

    state["phase"] = "VALIDATE"
    return state



###### ㄴ 4.2.3.3.4 Validator Action

In [ ]:
def fsm_validate(state: AgentState) -> AgentState:
    payload = {
        "user_question": state["user_question"],
        "plan": state["plan"],
        "tool_results": state["tool_results"],
        "draft_answer": state.get("draft_answer", ""),
    }
    resp = llm.invoke([VALIDATOR_SYS, HumanMessage(content=json.dumps(payload, ensure_ascii=False))])
    verdict_dict = safe_load_json(resp.content)
    verdict_obj = Verdict.model_validate(verdict_dict)

    state["verdict"] = verdict_obj.model_dump()
    state["trace"].append({"phase":"VALIDATE","ok":True,"verdict":state["verdict"]})

    if verdict_obj.ok:
        state["phase"] = "WRITE"
        return state

    # fail → replan 가능하면
    state["fix_hint"] = verdict_obj.fix_hint
    if state["replan_count"] < state["max_replans"]:
        state["phase"] = "REPLAN"
        state["trace"].append({"phase":"REPLAN_TRIGGER","replan_count":state["replan_count"]+1,"fix_hint":state["fix_hint"]})
        return state

    state["phase"] = "WRITE"  # 최후엔 write로
    return state

###### ㄴ 4.2.3.3.5 Replanner Action

In [ ]:
def fsm_replan(state: AgentState) -> AgentState:
    payload = {"user_question": state["user_question"], "fix_hint": state["fix_hint"]}
    msg = json.dumps(payload, ensure_ascii=False)

    try:
        # replan은 질문을 payload로 넣어서 안전하게
        plan_obj = make_plan_safe(msg, REPLANNER_SYS)
        state["plan"] = plan_obj.model_dump()
        state["replan_count"] += 1
        state["phase"] = "EXECUTE"
        state["trace"].append({"phase":"PLAN","ok":True,"used_replan_planner":True,"steps":state["plan"]["steps"]})
        return state
    except Exception as e:
        state["phase"] = "DONE"
        state["final"] = f"REPLAN_ERROR: {type(e).__name__}: {e}"
        state["trace"].append({"phase":"REPLAN","ok":False,"error":state["final"]})
        return state

###### ㄴ 4.2.3.3.6 Writer Action

In [ ]:
def fsm_write(state: AgentState) -> AgentState:
    payload = {
        "user_question": state["user_question"],
        "plan": state["plan"],
        "tool_results": state["tool_results"],
    }
    resp = llm.invoke([WRITER_SYS, HumanMessage(content=json.dumps(payload, ensure_ascii=False))])
    state["final"] = resp.content
    state["phase"] = "DONE"
    state["trace"].append({"phase":"WRITE","ok":True,"final_chars":len(state["final"])})
    return state

##### ㄴ 4.2.3.4 Transition Function

In [ ]:
from typing import Callable, Optional

Phase = str  # "PLAN" | "EXECUTE" | "VALIDATE" | "REPLAN" | "WRITE" | "DONE"

def transition(state: AgentState) -> AgentState:
    """FSM: advance exactly one step based on current phase."""
    phase = state["phase"]

    if phase == "PLAN":
        return fsm_plan(state)
    elif phase == "EXECUTE":
        return fsm_execute(state)
    elif phase == "VALIDATE":
        return fsm_validate(state)
    elif phase == "REPLAN":
        return fsm_replan(state)
    elif phase == "WRITE":
        return fsm_write(state)
    elif phase == "DONE":
        return state
    else:
        raise RuntimeError(f"Unknown phase: {phase}")

#### ㄴ 4.2.4 User Prompt

In [ ]:
question = "Plan-Then-Execute와 ReAct(LLM 에이전트 패턴)의 차이를 예시 포함해서 설명해줘. 최신 관점이면 search도 활용해."

#### ㄴ 4.2.5 Result

In [ ]:
def init_state(question: str, max_replans: int = 2) -> AgentState:
    return {
        "user_question": question,
        "phase": "PLAN",
        "plan": {},
        "tool_results": [],
        "verdict": {},
        "fix_hint": "",
        "replan_count": 0,
        "max_replans": max_replans,
        "draft_answer": "",
        "final": "",
        "trace": [],
    }

def run_loop(
    question: str,
    max_replans: int = 2,
    on_step: Optional[Callable[[AgentState, Phase], None]] = None,
) -> AgentState:
    """
    on_step(state, prev_phase): called after each transition.
    """
    state = init_state(question, max_replans=max_replans)

    while state["phase"] != "DONE":
        prev_phase = state["phase"]
        state = transition(state)

        if on_step:
            on_step(state, prev_phase)

    return state

In [ ]:
def run_fsm_graph(question: str, max_replans: int = 2) -> str:
    final_state = run_loop(question, max_replans=max_replans)
    return final_state["final"]

In [ ]:
print(run_fsm_graph(question, max_replans=2))

PLAN_ERROR: NameError: name 'llm' is not defined


### ㄴ 4.3 Verbose

In [ ]:
def print_execute_pretty(executed_nodes: List[str]):
    print("FSM: EXECUTE")
    print("  DAG: fan-out")
    for n in executed_nodes:
        if n.startswith("node_search_") or n.startswith("node_wiki_"):
            lane = n.split("_")[-1]
            print(f"    {n}({lane}) ...")

    print("  DAG: join")
    for n in executed_nodes:
        if n.startswith("node_merge_") or n == "node_compare":
            lane = n.split("_")[-1] if n.startswith("node_merge_") else ""
            suffix = f"({lane})" if lane else ""
            print(f"    {n}{suffix} ...")

    print("  DAG done → artifacts 저장")

def short(text: str, n: int = 40) -> str:
    if text is None:
        return ""
    text = str(text).replace("\n", " ").strip()
    return text[:n] + ("..." if len(text) > n else "")

def short(text: str, n: int = 40) -> str:
    if text is None:
        return ""
    text = str(text).replace("\n", " ").strip()
    return text[:n] + ("..." if len(text) > n else "")

def print_search_results(results, limit=3, width=40):
    # results: Tavily/Wiki 등의 list[dict] 형태를 가정
    for i, r in enumerate((results or [])[:limit]):
        title = r.get("title") or r.get("name") or ""
        content = r.get("content") or r.get("snippet") or r.get("summary") or ""
        print(f"      ({i}) {short(title, width)}")
        if content:
            print(f"           {short(content, width)}")

def print_tool_results_compact(tool_results, width=40, per_tool_limit=3):
    if not tool_results:
        print("TOOL RESULTS: (empty)")
        return

    print("TOOL RESULTS (compact):")
    for i, tr in enumerate(tool_results):
        tool = tr.get("tool", "")
        inp = tr.get("input", "")
        out = tr.get("output")

        print(f"  - [{i}] tool={tool} | input={short(inp, width)}")

        if isinstance(out, list):
            print_search_results(out, limit=per_tool_limit, width=width)
        else:
            print(f"      output={short(out, width)}")

In [ ]:
def verbose_hook(state: AgentState, prev_phase: Phase):
    print("\n" + "-"*28 + f" PHASE: {prev_phase} " + "-"*28)

    if prev_phase == "PLAN":
        print("GOAL:", state["plan"].get("goal"))
        print("STEPS:")
        for i, s in enumerate(state["plan"].get("steps", [])):
            print(f"  [{i}] {s['tool']} :: {s['input']}")

    elif prev_phase == "EXECUTE":
      exec_events = [x for x in state.get("trace", []) if x.get("phase") == "EXECUTE"]
      if not exec_events:
          print("Executed. (no EXECUTE trace found)")
          return

      t = exec_events[-1]
      executed_nodes = t.get("executed_nodes", [])

      # 1) 실행 노드 출력
      if executed_nodes:
          print_execute_pretty(executed_nodes)
      else:
          print("Executed steps:", len(state.get("tool_results", [])))

      # 2) tool_results를 40자 요약으로 출력
      print_tool_results_compact(state.get("tool_results", []), width=40, per_tool_limit=3)

    elif prev_phase == "VALIDATE":
        print("VERDICT:", state["verdict"])
        if not state["verdict"].get("ok", False):
            print("FIX_HINT:", state.get("fix_hint", ""))

    elif prev_phase == "REPLAN":
        print("Replanned. replan_count:", state["replan_count"])
        print("STEPS:")
        for i, s in enumerate(state["plan"].get("steps", [])):
            print(f"  [{i}] {s['tool']} :: {s['input']}")

    elif prev_phase == "WRITE":
        print("FINAL(trunc):")
        print(state["final"][:2000])


def run_fsm_graph_verbose(question: str, max_replans: int = 2) -> AgentState:
    print("\n" + "="*80)
    print("USER QUESTION:")
    print(question)

    final_state = run_loop(question, max_replans=max_replans, on_step=verbose_hook)

    print("\n" + "="*80)
    return final_state

In [ ]:
run_fsm_graph_verbose(question, max_replans=2)


USER QUESTION:
Plan-Then-Execute와 ReAct(LLM 에이전트 패턴)의 차이를 예시 포함해서 설명해줘. 최신 관점이면 search도 활용해.

---------------------------- PHASE: PLAN ----------------------------
GOAL: None
STEPS:



{'user_question': 'Plan-Then-Execute와 ReAct(LLM 에이전트 패턴)의 차이를 예시 포함해서 설명해줘. 최신 관점이면 search도 활용해.',
 'phase': 'DONE',
 'plan': {},
 'tool_results': [],
 'verdict': {},
 'fix_hint': '',
 'replan_count': 0,
 'max_replans': 2,
 'draft_answer': '',
 'final': "PLAN_ERROR: NameError: name 'llm' is not defined",
 'trace': [{'phase': 'PLAN',
   'ok': False,
   'error': "PLAN_ERROR: NameError: name 'llm' is not defined"}]}

## ㄴ 5. Role-Specialized Multi-Agent System


*   **System**이 **역할**을 **정의**
*   **LLM**은 **Role Function**으로 **호출**



### ㄴ 5.1 Description

하나의 LLM(싱글 에이전트)이 “기획/조사/구현/검수”를 다 하는 것과 달리

- **Planner**: 요구사항을 작업 단위로 분해, 실행계획 수립

- **Researcher**: 근거/레퍼런스/리스크 확인(선택)

- **Builder(Implementer)**: 실제 산출물(코드/설계/문서) 생성

- **Validator(Critic/QA)**: 산출물만 검사(형식/제약/정합성), 필요 시 수정 요청

역할별로 **system prompt + 책임 범위**를 분리하고,
상위 **Supervisor(Orchestrator)** 가 "지금 누구에게 넘길지" 라우팅

#### ㄴ 5.1.1 Architecture

```javascript    

                    ┌----------------------┐
                    │         USER         │
                    └----------┬----------┘
                                │
                                ▼
                   ┌-----------------------┐
                   │      SUPERVISOR       │   (LLM Router / Orchestrator)
                   │-----------------------│
                   │ reads Shared State    │
                   │ decides next action   │
                   │ stop / retry control  │
                   └----┬-----┬-----┬---┘
                         │     │     │
            ┌-----------┘     │     └-----------------┐
            │                  │                        │
            ▼                  ▼                        ▼
┌--------------------┐  ┌-------------------┐  ┌-------------------┐
│      PLANNER       │  │    RESEARCHER     │  │      CRITIC       │
│--------------------│  │-------------------│  │-------------------│
│ make plan +        │  │ gather evidence   │  │ evaluate draft    │
│ acceptance criteria│  │ using tools       │  │ vs criteria       │
│ OUTPUT: plan       │  │ OUTPUT: notes     │  │ OUTPUT: feedback  │
└---------┬---------┘  └--------┬---------┘  └---------┬--------┘
           │                       │                        │
           │                       │                        │
           ▼                       ▼                        ▼
┌----------------------------------------------------------------┐
│                           SHARED STATE                         │
│----------------------------------------------------------------│
│ goal / plan / criteria / notes / draft / feedback / approved   │
│ iter / max_iters / traces(tool results)                        │
└---------------------------┬-----------------------------------┘
                             │
                             │ (uses state inputs)
                             ▼
                   ┌-------------------┐
                   │      BUILDER      │
                   │-------------------│
                   │ produce artifact  │
                   │ apply feedback    │
                   │ tools (optional): │
                   │   - calculator    │
                   │ OUTPUT: draft     │
                   └---------┬--------┘
                              │
                              ▼
                           (back to SUPERVISOR)
```



##### ㄴ 5.1.1.1 Component

###### ㄴ **1) Supervisor (Orchestrator / Router)**
> 현재 Shared State를 읽고 "다음에 누구(어떤 역할 에이전트)를 호출할지" 를 결정

- **결정 포인트**
  - 지금 정보가 부족한가? → Researcher
  - 계획이 필요한가? → Planner
  - 산출물을 만들어야 하나? → Builder
  - 검수/품질 체크가 필요한가? → Critic/Validator
  - 종료 조건(approved, max_iter 등)을 만족했는가? → DONE
- **핵심**: 단일 에이전트처럼 한 번에 끝내지 않고, **흐름 제어(제어 plane)** 를 담당

###### ㄴ **2) Specialized Agents (Role-Specialized Workers)**
> 각자 좁은 책임 범위 + 필요 시, 도구 사용

- **Planner**
> 목표를 단계로 분해해서 plan 생성
  - Input: goal, constraints
  - Output: plan(단계/체크리스트/해야 할 일)
- **Researcher**
> wiki/search/docstore_lookup로 근거/정보 수집 → notes 생성
  - Input: goal, plan
  - Output: notes(필요한정보/가정/근거)
- **Builder**
> goal + plan + notes + feedback로 draft 생성 (필요 시 calculator로 수치 계산)
  - Input goal, plan, notes, feedback
  - Output: draft(초안 산출물)
- **Critic/Validator**
> draft를 평가해서 approved + feedback 생성 (지금은 안전/정확성/완성도 중심)
  - Input: draft + acceptance criteria
  - Output: review(approved 여부 + 수정 지시)

###### ㄴ **3) Shared State**
> **공유 작업공간**에 결과를 기록하고 읽음

- **Field**
  - goal: 사용자 목표
  - plan: 계획(steps)
  - notes: 조사 결과(근거/요약)
  - draft: 산출물 초안
  - approved: 통과 여부
  - feedback: 수정 지시
  - iter: 반복 횟수

###### ㄴ **4) Tools**
> 검색, 코드 실행, DB 조회, 파일 읽기 등

- Wikipedia Query / docstore_lookup
- Tavily Search
- calculator

##### ㄴ 5.1.1.2 Workflow


1.  User Goal 입력
2.  Supervisor → **Planner** (plan 생성)
3.  Supervisor → **Researcher** (tools 사용해서 notes 생성)
4.  Supervisor → **Builder** (draft 생성)
5.  Supervisor → **Critic** (approved/feedback)
6.  approved=false면 **Builder ↔ Critic** 1~2회 반복
7.  DONE (draft 반환)

### ㄴ 5.2 Code

#### ㄴ 5.2.1 dd

## ㄴ 6. Deterministic Guardrail Pattern

**Post-ReAct에서는 반드시 아래 항목이 존재**
*   Validation Step
*   Structured Output
*   JSON Contracts
*   Retry Policy

